# Bag of Words Meets Bags of Popcorn

## Data Preparation

In [1]:
import pandas as pd

df = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


Separate the features and the label.

In [3]:
y = df['sentiment']
X = df.drop('sentiment', axis=1)

## Data Wrangling

We will remove *id* column.

In [4]:
X.drop('id', axis=1, inplace=True)

Let's take a look at what a review looks like.

In [5]:
print(X['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

We will remove `<br />` tags, numbers and special characters.

In [6]:
X['review'] = X['review'].str.replace('<br />', ' ', regex=False)
X['review'] = X['review'].str.replace(r'[^A-Za-z]', ' ', regex=True)

Split the dataset into train and test parts.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

## Train Model

We are going to use a VADER lexicon. It is an unsupervised model, so we don't need any training.

In [8]:
import nltk

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/luke0201/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [9]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiment_intensity_analyzer = SentimentIntensityAnalyzer()

In [10]:
def vader_polarity(review, threshold=0.1):
    scores = sentiment_intensity_analyzer.polarity_scores(review)
    compound_score = scores['compound']
    sentiment = 1 if compound_score >= threshold else 0
    return sentiment

## Test Model

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

predictions = X_test['review'].apply(vader_polarity)
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print(f'Accuracy on the test dataset: {accuracy:.4f}')
print(f'Precision on the test dataset: {precision:.4f}')
print(f'Recall on the test dataset: {recall:.4f}')
print(f'F1 on the test dataset: {f1:.4f}')

Accuracy on the test dataset: 0.6958
Precision on the test dataset: 0.6457
Recall on the test dataset: 0.8593
F1 on the test dataset: 0.7373
